In [ ]:
# PROJECT AIM - Create an online booking system for using shared gym facilities. 
            # - Create a GUI using tkinter 
            # - Create a database using SQL to store the data entered by the user
            # - Return output messages within the GUI 

            
# Create a function to register as a user.
# Register as a user by entering your name, surname and flat number. 
# 1st check if all fields are inputted. 
# 2nd check if correct flat number format is entered.
# 3rd check if user has registered before. 
# 4th confirm user registration and allocate user ID. 



# Create a function to make a booking. 
# Make a booking by entering your preferred date/time and specified user_id 
# 1st check if all fields are inputted. 
# 2nd check if registered as a user. 
# 3rd check if correct date and time format entered.
# 4th check availability and confirm/reject booking. 


# FURTHER ACTIONS TO DO 
# 1. when checking availability, offer alternative times close to user input. 
# 2. limit the number of registered users to 2 per flat 
# 3. automate user input to test code 
# 4. limit time frame in booking function to one hour per booking 


In [11]:
#import necessary libraries

import sqlite3 
import os, sys
import tkinter as tk 
from tkinter import * 



# create database filepath 
DATABASE_FILEPATH = 'user_and_gym_bookings.db'

if os.path.exists(DATABASE_FILEPATH):
    os.remove(DATABASE_FILEPATH)
    

conn = sqlite3.connect(DATABASE_FILEPATH)
cur=conn.cursor()


cur.executescript("""

CREATE TABLE Users(
user_id INTEGER PRIMARY KEY,
name VARCHAR (200) NOT NULL, 
surname VARCHAR (200) NOT NULL, 
flat_number INT NOT NULL, 
CONSTRAINT UC_Users UNIQUE (name, surname, flat_number)
);

CREATE TABLE Bookings(
booking_id INTEGER PRIMARY KEY, 
booking_date DATE NOT NULL, 
start_time TIME NULL, 
end_time TIME NULL, 
user_id INTEGER,
CONSTRAINT UC_Bookings UNIQUE (booking_date, start_time, end_time)
)

""")

#create function to register as a user
def reg_user():
    #create a database or connect to one 
    conn=sqlite3.connect(DATABASE_FILEPATH)
    #create a cursor 
    cur=conn.cursor()
    
    #check if inputs are empty, print an error statement 
    if ent_name.get()=='' or ent_surname.get() =='' or ent_flat_number.get() =='':
        answer.config(text='Please complete all fields.')    
    #check if an integer is entered for the flat number 
    else:  
        try:
            int(ent_flat_number.get())
          
    #check if user has previously registered
            cur.execute('''SELECT name, surname, flat_number FROM Users WHERE name =:name AND surname=:surname AND flat_number=:flat_number''',
               {
                   'name':ent_name.get(),
                   'surname':ent_surname.get(),
                   'flat_number':ent_flat_number.get()
               })
            result=cur.fetchone()
            #print(result)
     #if user input is NOT a duplicate, add input to database 
            if not result:
                cur.execute('''INSERT INTO Users (name, surname, flat_number) VALUES (:name, :surname, :flat_number)''',
                   {
                   'name':ent_name.get(),
                   'surname':ent_surname.get(),
                   'flat_number':ent_flat_number.get()
                   })
     #allocate a user ID upon registration
                cur.execute('''SELECT user_id FROM Users WHERE name=:name AND surname=:surname AND flat_number=:flat_number''',
                    {
                   'name':ent_name.get(),
                   'surname':ent_surname.get(),
                   'flat_number':ent_flat_number.get()
                       })
                user_id=cur.fetchone()[0]
                reg_comp='Thank you for registering ' + ent_name.get() + '. Your user ID is, ' + str(user_id) + '. You can now make a booking.'
                answer.config(text=reg_comp)
                #print(user_id)
                
      #otherwise if data input has been previously entered, print a statement     
            else: 
                answer.config(text='You have previously registered as a user.')
            
        except ValueError:
                answer.config(text='Please enter a valid flat number in digits.')
                
        #delete data input 
        ent_name.delete(0,END)
        ent_surname.delete(0,END)
        ent_flat_number.delete(0, END)
        
        #commit changes
        conn.commit()
        #close connection to database
        conn.close()
        
####################################### BREAK #################################################   

        
#create function to make a booking 
from datetime import * 
def make_booking():
    
    #create a database or connect to one 
    conn=sqlite3.connect(DATABASE_FILEPATH)
    #create a cursor 
    cur=conn.cursor()       
    #check if all fields are completed 
    if ent_user_id.get()=='' or ent_booking_date.get()=='' or ent_start_time.get()=='' or ent_end_time.get()=='':
        answer.config(text='Please complete all fields to make a booking.')
    
    else: 
        
            
            cur.execute('''SELECT user_id FROM Users WHERE EXISTS (SELECT user_id FROM Users WHERE user_id=:user_id)''',
               {
                   'user_id':ent_user_id.get(),
               })
           
            result=cur.fetchone()
            #print(result)
            
            
                     
            if result: 
                try:
                    datetime.strptime(ent_booking_date.get(), '%d-%m-%Y') and datetime.strptime(ent_start_time.get(), '%H:%M') and datetime.strptime(ent_end_time.get(), '%H:%M') 

                
                
                    cur.execute('''SELECT booking_date, start_time, end_time FROM Bookings WHERE EXISTS (SELECT booking_date, start_time, end_time FROM Bookings WHERE booking_date=:booking_date AND start_time=:start_time AND end_time=:end_time)''',
                        {
                            'booking_date':ent_booking_date.get(),
                            'start_time':ent_start_time.get(),
                            'end_time':ent_end_time.get(),
                            
                        })
                    result2=cur.fetchall()
                    #print(result2)
                    
                    if not result2: 
                        
                    
                        cur.execute('''INSERT INTO Bookings (booking_date, start_time, end_time, user_id) VALUES (:booking_date, :start_time, :end_time, :user_id)''',
                               {
                                'booking_date':ent_booking_date.get(),
                                'start_time':ent_start_time.get(),
                                'end_time':ent_end_time.get(),
                                'user_id':ent_user_id.get(),
                               })

                        answer.config(text='Thank you for booking.') 
                    
                    if result2: 
                        answer.config(text='Please enter an alternative date and time.')

                    
                except: 
                    answer.config(text='Please enter date and time in correct format.')
            
            if not result:
                 answer.config(text='Please register as a user first.')
                
                
        
            
       
    #delete data input 
    ent_booking_date.delete(0,END)
    ent_start_time.delete(0,END)
    ent_end_time.delete(0,END)
    ent_user_id.delete(0,END)
        

        
    #commit changes
    conn.commit()
    #close connection to database
    conn.close()
    


    
#create tkinter GUI 

import tkinter as tk 
from tkinter import * 

#create window
window=tk.Tk()
window.title('Welcome to the bookings page')

#create text boxes for user input
ent_name=tk.Entry(width=30)
ent_name.grid(row=0, column=1, padx=20)

ent_surname=tk.Entry(width=30)
ent_surname.grid(row=1, column=1, padx=20)

ent_flat_number=tk.Entry(width=30)
ent_flat_number.grid(row=2, column=1, padx=20)

ent_booking_date=tk.Entry(width=30)
ent_booking_date.grid(row=4,column=1, padx=20)

ent_start_time=tk.Entry(width=30)
ent_start_time.grid(row=5,column=1, padx=20)

ent_end_time=tk.Entry(width=30)
ent_end_time.grid(row=6,column=1, padx=20)

ent_user_id=tk.Entry(width=30)
ent_user_id.grid(row=7, column=1, padx=20)

#create label widget for text boxes 
lbl_name=tk.Label(text='Name')
lbl_name.grid(row=0, column=0)

lbl_surname=tk.Label(text='Surname')
lbl_surname.grid(row=1, column=0)

lbl_flat_number=tk.Label(text='Flat Number')
lbl_flat_number.grid(row=2, column=0)

lbl_booking_date=tk.Label(text='Booking Date')
lbl_booking_date.grid(row=4, column=0)

lbl_start_time=tk.Label(text='Start Time')
lbl_start_time.grid(row=5, column=0)

lbl_end_time=tk.Label(text='End Time')
lbl_end_time.grid(row=6, column=0)

lbl_user_id=tk.Label(text='User ID')
lbl_user_id.grid(row=7, column=0)


#create a register user button
reg_btn=tk.Button(text='Register', command=reg_user)
reg_btn.grid(row=3, columnspan=2, padx=10, pady=10)

book_btn=tk.Button(text='Book', command =make_booking)
book_btn.grid(row=9, columnspan=2, padx=10, pady=10)


#create an answer in the GUI
answer=tk.Label(text='')
answer.grid(row=10, column=0)

cur.close()
conn.commit()
conn.close() 


window.mainloop() 


